# Побудова оцінок та довірчих інтервалів

Будуємо довірчі інтервали для `n = 100, 10_000, 1_000_000`
Рівень довіри = `0,99` (рівень значущості = `0.01`

#### Довірчі інтервали для м.с. та дисперсії у випадку нормально розподіленої генеральної сукупності.
##### Перша частина леми 1. (Пункт А):
Вибіркове математичне сподівання: $\hat{a}_{n}=\frac {1}{n}\sum^{n}_{k=1}X_k$
Вибіркова дисперсія: $\hat{\sigma}^2_{n}=\frac {1}{n}\sum^{n}_{k=1}(X_k-\hat{a}_{n})^2$
Дов інтервал $a \in a_n \pm \frac{z_{\gamma}  \sigma_n}{\sqrt{n}}$
К-ть ітерацій: $n$

In [72]:
import math

import numpy as np
from scipy.stats import chi2, t, norm

In [73]:
def calc_sample_variance(samples, sample_mean: float):
    squared_variance = 0.
    for sample in samples:
        squared_variance += (sample - sample_mean) ** 2

    squared_variance /= (len(samples) - 1)
    return math.sqrt(squared_variance)


def calc_sample_mean(samples):
    mean = 0.
    for sample in samples:
        mean += sample
    return mean / len(samples)


def calc_s_squared(samples, sample_mean):
    s_squared = 0
    for sample in samples:
        s_squared += (sample - sample_mean) ** 2

    s_squared /= (len(samples) - 1)
    return s_squared


# generating samples with help of N(0,1) distribution
def gen_samples(size: int):
    samples = size * [None]
    for i in range(size):
        samples[i] = np.random.normal(0, 1)
    return samples

In [74]:
n_arr = [100, 10_000, 1_000_000]  # for calculating Confidence Intervals
m_arr = [1, 10, 100, 1_000, 10_000]  # for calculating Q_m

samples_list = len(n_arr) * [None]
alfa = 0.01
gama = 1 - alfa
z_gama = 2.575  # norm distro 0.99 quantile

Q_m = lambda q, n: np.sum(q) / n
# calculating variance in integral methods
# незміщена?
calc_variance = lambda q, n, Q: (1 / (n-1)) * (np.sum([qi**2 for qi in q]) - n * Q ** 2)

eta = lambda x: 1 / x - 1
xi_i = lambda x: -1 * np.log(x)

# calculating 0.99 quantiles
print(f"samples_num_ci:{n_arr}\nm_arr:{m_arr}")

samples_num_ci:[100, 10000, 1000000]
m_arr:[1, 10, 100, 1000, 10000]


In [75]:
# calculating Confidence Interval for the EXPECTATION(sigma^2) under the assumption:
# - observed random value ~ N(0,1)
# - sample variance is unknown
def calculate_ci_1(samples):
    sample_mean = calc_sample_mean(samples)
    n_sq_root = math.sqrt(len(samples))
    s_squared = calc_s_squared(samples, sample_mean)

    g = t.ppf((1 + gama) / 2, df=len(samples) - 1)
    return [sample_mean - math.sqrt(s_squared) * g / n_sq_root, sample_mean + math.sqrt(s_squared) * g / n_sq_root]


# calculating Confidence Interval for the EXPECTATION under assumption:
# - observed value distribution is unknown
# asymptotic confidence interval
def calculate_ci_2(samples, gama):
    sample_mean = calc_sample_mean(samples)
    sample_variance = calc_sample_variance(samples, sample_mean)

    n_sq_root = math.sqrt(len(samples))

    c = norm.ppf((1 + gama) / 2)
    return [sample_mean - sample_variance * c / n_sq_root, sample_mean + sample_variance * c / n_sq_root]


# calculating Confidence Interval for the variance under assumption:
# - observed random value ~ N(0,1)
def calculate_ci_3(samples, gama):
    sample_mean = calc_sample_mean(samples)
    g_1 = chi2.ppf((1 - gama) / 2, df=len(samples) - 1)
    g_2 = chi2.ppf((1 + gama) / 2, df=len(samples) - 1)

    squared_difference = 0
    for sample in samples:
        squared_difference = (sample - sample_mean) ** 2

    return [squared_difference / g_2, squared_difference / g_1]


In [76]:

def calculate_2_integral():
    pass

In [77]:
def calculate_3_integral():
    pass


In [78]:
for i, sample_size in enumerate(n_arr):
    samples_list[i] = gen_samples(sample_size)
    sample_mean = calc_sample_mean(samples_list[i])
    sample_variance = calc_sample_variance(samples_list[i], sample_mean)

    ci_1 = calculate_ci_1(samples_list[i])
    print('\n[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркове математичне сподівання: {sample_mean}')
    print(f'Довірчий інтервал:[{ci_1[0]} ; {ci_1[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_1[1] - ci_1[0]}')

    ci_2 = calculate_ci_2(samples_list[i], gama)
    print('\n[2] Довірчий інтервал для мат. спод. M при невідомому розподілі')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркове математичне сподівання: {sample_mean}')
    print(f'Довірчий інтервал:[{ci_2[0]} ; {ci_2[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_2[1] - ci_2[0]}')

    ci_3 = calculate_ci_3(samples_list[i], gama)
    print('\n[3] Lовірчий інтервал для дисперсії')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркова дисперсія = {sample_variance}')
    print(f'Довірчий інтервал:[{ci_3[0]} ; {ci_3[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_3[1] - ci_3[0]}')


[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії
Кількість спостережень: n = 100
Вибіркове математичне сподівання: 0.002592152743340701
Довірчий інтервал:[-0.27747308089755524 ; 0.2826573863842366]
Довжина довірчого інтервалу: 0.5601304672817918

[2] Довірчий інтервал для мат. спод. M при невідомому розподілі
Кількість спостережень: n = 100
Вибіркове математичне сподівання: 0.002592152743340701
Довірчий інтервал:[-0.27207992195999836 ; 0.2772642274466797]
Довжина довірчого інтервалу: 0.5493441494066781

[3] Lовірчий інтервал для дисперсії
Кількість спостережень: n = 100
Вибіркова дисперсія = 1.0663442423180143
Довірчий інтервал:[0.01625030668347501 ; 0.03395841648093884]
Довжина довірчого інтервалу: 0.017708109797463834

[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії
Кількість спостережень: n = 10000
Вибіркове математичне сподівання: -0.006822308416369051
Довірчий інтервал:[-0.032488220130919294 ; 0.01884360329818119]
Довжина довірчого інтервалу: 0.051331

# Метод 1
Будемо обраховувати інтеграл за допомогою методу Монте-Карло:
$$Q_m = \bf{M}(I(\eta > \xi_{1} + \ldots + \xi_m))$$

(варто підмітити, що із збільшенням m значення інтеграла зменшується у $\sqrt{m}$ разів)

In [79]:
q_1 = lambda eta, samples: 1 if eta > np.sum(samples) else 0
m_arr = [1, 10, 100, 1_000]  # for calculating Q_m
n_0 = 5000  #

for m in m_arr:
    indicators = []

    for j in range(n_0):
        samples = np.random.random(m)
        indicators.append(q_1(eta(np.random.random(1)), samples))

    Q = Q_m(indicators, len(indicators))
    sigma = calc_variance(indicators, len(indicators),  Q)

    print(f"Кількість зразків: {m}")
    print(f"Оцінка Q: {Q}")
    print(f"Вибіркова дисперсія: {sigma}")
    print(f"Асимптотичний довірчий інтервал: {calculate_ci_2(indicators, gama)}")
    print(f"Кількість спостережень: {n_0}")
    print(f"N*: {(2.575 * 100) ** 2 * sigma / (Q ** 2)}\n")

Кількість зразків: 1
Оцінка Q: 0.687
Вибіркова дисперсія: 0.21507401480296054
Асимптотичний довірчий інтервал: [0.6701062505654658, 0.7038937494345343]
Кількість спостережень: 5000
N*: 30215.440831980068

Кількість зразків: 10
Оцінка Q: 0.1724
Вибіркова дисперсія: 0.14270678135627127
Асимптотичний довірчий інтервал: [0.15863886095874777, 0.18616113904125223]
Кількість спостережень: 5000
N*: 318364.4414497749

Кількість зразків: 100
Оцінка Q: 0.0194
Вибіркова дисперсія: 0.01902744548909782
Асимптотичний довірчий інтервал: [0.014375159039298192, 0.02442484096070181]
Кількість спостережень: 5000
N*: 3352212.1305704433

Кількість зразків: 1000
Оцінка Q: 0.0028
Вибіркова дисперсія: 0.002792718543708742
Асимптотичний довірчий інтервал: [0.0008749338310975875, 0.004725066168902413]
Кількість спостережень: 5000
N*: 23619221.165661708



In [80]:

m_arr = [1, 10, 100, 1_000, 10_000]  # for calculating Q_m
n_0 = 100


In [81]:
m_arr = [1, 10, 100, 1_000, 10_000]  # for calculating Q_m
n_0 = 10

